# Projeto 1 - Ciência dos Dados

Nome: Raphael Lahiry de Barros

Nome: Florencia Averame Kramer

# O que temos que fazer?
precisamos criar um dataframe para cada relevância   OK

dentro desses dataframes separar as frases em palavras OK   

analisar a freq relativa de cada palavra --> probabilidade daquela palavra pertencer àquela relevância    

quando quisermos avaliar o teste--> analisamos a probabilidade de cada palavra de uma frase pertencer a uma relevância (usar o smoothing -- aula 8 -- para as palavras que talvez não existam em certas categorias)  

com isso, para cada palavra teremos a probabilidade de que esta pertença a cada relevância  
a relevância com maior probabilidade será a daquela palavra

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
## Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [4]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo ps4.xlsx, tudo certo para prosseguir com a prova!


### Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [5]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,@odonodocabare_ comprei um tablet dia desses e...,0.0
1,@encanadorchapad o negócio é o ps4 slim. peque...,4.0
2,rt @ps3brasil: tohu é anunciado para ps4; deta...,2.0
3,dividindo essa semana entre dois jogos no modo...,0.0
4,@leobruto @fridaygus @douglasxavier8 @ocerveje...,0.0


In [6]:
train['Treinamento'] = train['Treinamento'].astype('str')

In [7]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@chaehobbit simmm sou apaixonada em jogos assi...,1.0
1,"@pimps_junin @joao_almirante atleta nato , pod...",1.0
2,@isa_boechatt kkkk vc joga onde no xbox ps4 ou...,1.0
3,@vezguu eu sou ps4,1.0
4,path of exile (ps4) - farmando e fazendo o atl...,0.0


In [8]:
test['Teste'] = test['Teste'].astype('str')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é o playstation 4. Consideramos relevantes as mensagens que falavam sobre questões técnicas do produto e sobre satisfação, sejam positivas ou negativas. Consideramos irrelevantes as mensagens que só mencionavam o nosso produtos nas '#' ou que eram opiniões pessoais que não agregavam na nossa análise.

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [9]:
#função que apaga os caracteres especiais e as menções de usuários
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;[]]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    #filtro que remove as menções à usuários com @
    filtro = filter(lambda x:x[0]!='@', text_subbed.split())
    juncao = " ".join(filter(lambda x:x[0]!='@', text_subbed.split()))
    
    return juncao
    

In [10]:
#aplicando a função cleanup na base treinamento e deixando tudo em letra minúscula
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train['Treinamento'] = train['Treinamento'].str.lower()

In [11]:
train['Relevância'] = train['Relevância'].astype('category')

In [12]:
train['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']
train['Relevância'].cat.as_ordered()

0      Muito irrelevante
1        Muito relevante
2                 Neutro
3      Muito irrelevante
4      Muito irrelevante
             ...        
746          Irrelevante
747          Irrelevante
748               Neutro
749    Muito irrelevante
750          Irrelevante
Name: Relevância, Length: 751, dtype: category
Categories (5, object): [Muito irrelevante < Irrelevante < Neutro < Relevante < Muito relevante]

In [13]:
#ligando a relevância à um número
muito_relevante = train['Relevância'] == 'Muito relevante'
relevante = train['Relevância'] == 'Relevante'
neutro = train['Relevância'] == 'Neutro'
irrelevante = train['Relevância'] == 'Irrelevante'
muito_irrelevante = train['Relevância'] == 'Muito irrelevante'

In [14]:
train.head()

,Treinamento,Relevância
0,comprei um tablet dia desses e to bem feliz......,Muito irrelevante
1,"o negócio é o ps4 slim. pequeno e silencioso, ...",Muito relevante
2,rt tohu é anunciado para ps4; detalhes e gamep...,Neutro
3,dividindo essa semana entre dois jogos no modo...,Muito irrelevante
4,sentiu ? kkkkk. só fiz uma pergunta bem simple...,Muito irrelevante


### Filtros para criar os dataFrames com cada relevância

In [15]:
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_neutro = train.loc[neutro, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]

### Separando cada coluna de relevância em listas de palavras

In [16]:
palavras_muito_relevantes = []
lista_vrw = []
for vrw in df_muito_relevante['Treinamento']:
    palavras_muito_relevantes.append(vrw.split(' '))
for vrl in palavras_muito_relevantes:
    lista_vrw += vrl

In [17]:
palavras_relevantes = []
lista_rw = []
for rw in df_relevante['Treinamento']:
    palavras_relevantes.append(rw.split(' '))
for rl in palavras_relevantes:
    lista_rw += rl

In [18]:
palavras_neutras = []
lista_nw = []
for nw in df_neutro['Treinamento']:
    palavras_neutras.append(nw.split(' '))
for nl in palavras_neutras:
    lista_nw += nl

In [19]:
palavras_irrelevantes = []
lista_iw = []
for iw in df_irrelevante['Treinamento']:
    palavras_irrelevantes.append(iw.split(' '))
for il in palavras_irrelevantes:
    lista_iw += il

In [20]:
palavras_muito_irrelevantes = []
lista_viw = []
for viw in df_muito_irrelevante['Treinamento']:
    palavras_muito_irrelevantes.append(viw.split(' '))
for vil in palavras_muito_irrelevantes:
    lista_viw += vil

### Transformando as listas de palavras de cada relevância em pd.Series

In [21]:
serie_muito_relevante = pd.Series(lista_vrw)

In [22]:
serie_relevante = pd.Series(lista_rw)

In [23]:
serie_neutro = pd.Series(lista_nw)

In [24]:
serie_irrelevante = pd.Series(lista_iw)

In [25]:
serie_muito_irrelevante = pd.Series(lista_viw)

### Frequências relativas e absolutas das palavras de cada relevância

In [132]:
freq_ab_muito_relevante = serie_muito_relevante.value_counts() 
freq_re_muito_relevante = serie_muito_relevante.value_counts(True) 

print(freq_re_muito_relevante['igual'])

0.0008237232289950577


In [133]:
freq_ab_relevante = serie_relevante.value_counts()
freq_re_relevante = serie_relevante.value_counts(True)

print(freq_re_relevante['igual'])

KeyError: 'igual'

In [134]:
freq_ab_neutro = serie_neutro.value_counts()
freq_re_neutro = serie_neutro.value_counts(True)

print(freq_re_neutro['igual'])

0.00028546959748786756


In [135]:
freq_ab_irrelevante = serie_irrelevante.value_counts()
freq_re_irrelevante = serie_irrelevante.value_counts(True)

print(freq_re_irrelevante['igual'])

0.0005003752814610958


In [136]:
freq_ab_muito_irrelevante = serie_muito_irrelevante.value_counts()
freq_re_muito_irrelevante = serie_muito_irrelevante.value_counts(True)

print(freq_re_muito_irrelevante['igual'])

0.0006253908692933083


### Definindo a quantidade total de palavras na língua portuguesa

In [31]:
#portugues são as palavras existes que a gente vai verificar a freq relativa e vamos somar um alpha pra garantir
#que as palavras que não existem não zerem o classificador
total_train = lista_vrw + lista_rw + lista_nw + lista_iw + lista_viw

#na fórmula precisamos dividir pela quantidade de palavras na lingua portuguesa
alpha = 1
v = 1e6
lingua_portuguesa = len(total_train) + alpha*v

### Pd.series do português

In [32]:
serie_total_train = pd.Series(total_train)
serie_total_train.value_counts()

ps4                                 545
o                                   463
de                                  369
e                                   359
no                                  274
                                   ... 
vaga                                  1
otimos                                1
legenda                               1
&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;      1
lanche                                1
Length: 4270, dtype: int64

### Criando a probabilidade de cada relevância à priori

In [33]:
p_mr = len(serie_muito_relevante)/len(serie_total_train)
p_r = len(serie_relevante)/len(serie_total_train)
p_n = len(serie_neutro)/len(serie_total_train)
p_i = len(serie_irrelevante)/len(serie_total_train)
p_mi = len(serie_muito_irrelevante)/len(serie_total_train)

print(p_mr + p_r + p_n + p_i + p_mi)

1.0


## Montando Naive-Bayes

### Classificador da relevância de cada palavra

In [130]:

# lista muito relevante
dict_relevancia_palavras = {}


for palavra in total_train:
    if (palavra not in lista_vrw) and (palavra not in lista_rw) and (palavra not in lista_nw) and (palavra not in lista_iw) and (palavra not in lista_viw):
        mr = alpha/len(total_train)
        r = alpha/len(total_train)
        n = alpha/len(total_train)
        i = alpha/len(total_train)
        mi = alpha/len(total_train)
        
    if palavra in lista_vrw:
        mr = freq_re_muito_relevante[palavra] 
       
    if palavra in lista_rw:
        r = freq_re_relevante[palavra] 
       
    if palavra in lista_nw:
        n = freq_re_neutro[palavra] 
        
    if palavra in lista_iw:
        i = freq_re_irrelevante[palavra] 
       
    if palavra in lista_viw:
        mi = freq_re_muito_irrelevante[palavra] 
     
    if (mr > r) and (mr > n) and (mr > i) and (mr > mi):
        #print('mr')
        dict_relevancia_palavras[palavra] = 4
        #break
    elif (r > mr) and (r > n) and (r > i) and (r > mi):
        #print('r')
        dict_relevancia_palavras[palavra] = 3
        #break
    elif (n > mr) and (n > r) and (n > i) and (n > mi):
        #print('n')
        dict_relevancia_palavras[palavra] = 2
        #break
    elif (i > mr) and (i > r) and (i > n) and (i > mi):
        #print('i')
        dict_relevancia_palavras[palavra] = 1
        #break
    elif (mi > mr) and (mi > r) and (mi > n) and (mi > i):
        #print('mi')
        dict_relevancia_palavras[palavra] = 0
        #break
    
dict_relevancia_palavras['igual']

3

In [131]:
# lista muito relevante

def classificador_relevancia(palavra):
    
    dict_relevancia_palavras = {}
    
    if palavra not in total_train:
        dict_relevancia_palavras[palavra] = alpha/lingua_portuguesa
        print('NÂO EXISTIA DENTRO DE TRAIN')
        
    else:
        print("ENTROU NO ELSE")
        mr=0
        r=0
        n=0
        i=0
        mi=0
            
        if palavra in lista_vrw:
            mr = freq_re_muito_relevante[palavra] + (alpha/lingua_portuguesa)

        if palavra in lista_rw:
            r = freq_re_relevante[palavra] + (alpha/lingua_portuguesa) 

        if palavra in lista_nw:
            n = freq_re_neutro[palavra] + (alpha/lingua_portuguesa) 

        if palavra in lista_iw:
            i = freq_re_irrelevante[palavra] + (alpha/lingua_portuguesa) 

        if palavra in lista_viw:
            mi = freq_re_muito_irrelevante[palavra] + (alpha/lingua_portuguesa) 

        if (mr > r) and (mr > n) and (mr > i) and (mr > mi):
            print('mr')
            dict_relevancia_palavras[palavra] = 4

        elif (r > mr) and (r > n) and (r > i) and (r > mi):
            print('r')
            dict_relevancia_palavras[palavra] = 3

        elif (n > mr) and (n > r) and (n > i) and (n > mi):
            print('n')
            dict_relevancia_palavras[palavra] = 2

        elif (i > mr) and (i > r) and (i > n) and (i > mi):
            print('i')
            dict_relevancia_palavras[palavra] = 1

        elif (mi > mr) and (mi > r) and (mi > n) and (mi > i):
            print('mi')
            dict_relevancia_palavras[palavra] = 0

    return dict_relevancia_palavras[palavra]

classificador_relevancia('igual')

ENTROU NO ELSE
mr


4

In [55]:
# criar a função que retorna a probabilidade da frase pertencer à uma cateogria específica de relevância com base nas palavras que à compõem

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [36]:
test['Teste'] = test['Teste'].apply(cleanup)
test['Teste'] = test['Teste'].str.lower()

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**